In [1]:
import sys
sys.path.append("../../lartpc_mlreco3d")
sys.path.append("../../pi0_reco")

import yaml
# Configuration for the data loader
io_cfg = """
iotool:
  batch_size: 1
  shuffle: False
  num_workers: 1
  collate_fn: CollateSparse
  dataset:
    name: LArCVDataset
    data_keys:
     #- /gpfs/slac/staas/fs1/g/neutrino/kterao/data/mpvmpr_2020_01_v04/train.root
     - /home/rberner/cernbox/PhD/pi0_reconstruction/reco_software/data/train.root
    limit_num_files: 25
    schema:
      input_data:
        - parse_sparse3d_scn
        - sparse3d_pcluster
      segment_label:
        - parse_sparse3d_scn
        - sparse3d_pcluster_semantics
      cluster_label:
        - parse_cluster3d_full
        - cluster3d_pcluster
        - particle_corrected
      particles:
        - parse_particle_asis
        - particle_corrected
        - cluster3d_pcluster
      ppn_label:
        - parse_particle_points
        - sparse3d_pcluster
        - particle_corrected
"""

In [2]:
# Configuration of the reconstruction chain
chain_cfg = '''
name: pi0_chain_test
net_cfg: /home/rberner/cernbox/PhD/pi0_reconstruction/reco_software/config_files/uresnet_ppn.cfg
segment:               label  # label, uresnet
deghost:                      # label, uresnet
charge2energy:         null # null, constant, average(, full, enet)
charge2energy_cst:     0.0082 # energy response constant (0.0082 for mask, 0.0052 for uresnet)
charge2energy_average: 0.877  # energy response average (1.3693 for mask, 0.877 for uresnet)
shower_start:          label  # label, ppn
shower_fragment:       label  # label, dbscan
shower_direction:      label  # label, pca, cent
shower_cluster:        label  # label, cone
shower_cluster_params:
  IP: 40
  Distance: 300
shower_energy:         pixel_sum  # label, pixel_sum
shower_match:          label  # label, proximity
refit_dir:             true   # true, false
refit_cone:            true   # true, false
'''
chain_cfg = yaml.load(chain_cfg,Loader=yaml.Loader)

In [3]:
from IPython.display import HTML, display

# Decorative progress bar
def progress(count, total, unit, message=''):
    return HTML("""
        <progress 
            value='{count}'
            max='{total}',
            style='width: 30%'
        >
            {count}
        </progress> {count}/{total} {unit} ({frac}%) ... {message}
    """.format(count=count, total=total, unit=unit, frac=int(float(count)/float(total)*100.),message=message))

In [4]:
# Initialize the chain
from pi0.chain import Pi0Chain
chain = Pi0Chain(io_cfg, chain_cfg)
from larcv import larcv
import numpy as np
from numpy import linalg

# Loop over the full dataset
masses = []
i_draw, n_draw = 0, 1
data_size = 5 #len(chain.hs.data_io) # len(chain.hs.data_io) = 125480
out = display(progress(0,data_size,'images'),display_id=True)

# Create CSV file for all true pi0s in the event
from mlreco.utils import CSVData
log_path_true_pi0 = 'true_pi0s.csv'
log_true_pi0 = CSVData(log_path_true_pi0)

# Create CSV file for all groups (group = collection of clusters) in the event
#from mlreco.utils import CSVData
#log_path_showers = 'showers.csv'
#log_showers = CSVData(log_path_showers)

for ev in range(data_size):
    #if ev != 85:
    #    continue
    #if ev < 80:
    #    continue
    print(' event: ', ev)

    # Loop over events and log informations in .csv file
    # ------------------------------------------------------
    chain.run_loop()

    #print(' ====================== ============== ======================')
    #print(' ======================   TRUE  INFO   ======================')
    #print(' ====================== ============== ======================')
    # Loop over all particles in the event and obtain true pi0->gamma+gamma info:
    # Note: primary pi0 are not stored in chain.event['particles']
    n_pi0_in_event           = 0   # [-]
    n_true_gammas            = 0   # [-]
    n_reco_gammas            = 0   # [-]
    pi0_track_ids            = []  # [-]
    pi0_pdg_code             = []  # [-]
    true_gammas_group_ids    = []  # [-]
    gamma_1_track_id         = []  # [-]
    gamma_1_pdg_code         = []  # [-]
    gamma_1_mom              = []  # [MeV/c]
    gamma_1_first_step       = []  # [x, y, z]
    gamma_1_pos              = []  # [x, y, z]
    gamma_1_ekin             = []  # [MeV]
    gamma_1_energy_deposit   = []  # [MeV]
    gamma_1_num_voxels       = []  # [-]
    gamma_2_track_id         = []  # [-]
    gamma_2_pdg_code         = []  # [-]
    gamma_2_mom              = []  # [MeV/c]
    gamma_2_first_step       = []  # [x, y, z]
    gamma_2_pos              = []  # [x, y, z]
    gamma_2_ekin             = []  # [MeV]
    gamma_2_energy_deposit   = []  # [MeV]
    gamma_2_num_voxels       = []  # [-]


    for particle in range(len(chain.event['particles'][0])):
        p = chain.event['particles'][0][particle]
        if p.parent_pdg_code() == 111 and p.pdg_code() == 22:
            n_true_gammas += 1
            if p.group_id() not in true_gammas_group_ids:
                true_gammas_group_ids.append(p.group_id())

            #print('\n----------------------------------------------- GENERAL ------------------------------------------------')
            #print(' Event Nr.: \t \t \t ', ev, '\n')
            #print(p.dump())
            
            '''
            print('\n----------------------------------------------- ANCESTOR -----------------------------------------------')
            print(' ancestor_track_id: \t \t ',      p.ancestor_track_id())
            print(' ancestor_pdg_code: \t \t ',      p.ancestor_pdg_code())
            print(' ancestor_creation_process: \t ', p.ancestor_creation_process())
            print(' ancestor_position: \t \t ',      p.ancestor_position().x(), p.ancestor_position().y(), p.ancestor_position().z())
            print(' ancestor_t: \t \t \t ',          p.ancestor_t())
            print(' ancestor x,y,z: \t \t ',         p.ancestor_x(), p.ancestor_y(), p.ancestor_z())
            print('\n------------------------------------------------ PARENT ------------------------------------------------')
            print(' parent_track_id: \t \t ',        p.parent_track_id())
            print(' parent_id: \t \t \t ',           p.parent_id())
            print(' parent_pdg_code: \t \t ',        p.parent_pdg_code())
            print(' parent_creation_process: \t ',   p.parent_creation_process())
            ##print(' parent_position: \t \t  (',      p.parent_position().x(), ' , ', p.parent_position().y(), ' , ', p.parent_position().z(), ')')   # This probably is not meaningful for primary pi0 (is it filled properly?)
            ##print(' parent_t: \t \t \t ',            p.parent_t())   # This probably is not meaningful for primary pi0 (is it filled properly?)
            ##print(' parent_x: \t \t \t ',            p.parent_x())   # This probably is not meaningful for primary pi0 (is it filled properly?)
            ##print(' parent_y: \t \t \t ',            p.parent_y())   # This probably is not meaningful for primary pi0 (is it filled properly?)
            ##print(' parent_z: \t \t \t ',            p.parent_z())   # This probably is not meaningful for primary pi0 (is it filled properly?)
            '''
            '''
            print('\n------------------------------------------------ PHOTON ------------------------------------------------')
            #print(' boundingbox_2d: ', p.boundingbox_2d())
            #print(' boundingbox_3d: ', p.boundingbox_3d())
            print(' track_id: \t \t \t ',            p.track_id())
            print(' id: \t \t \t \t ',               p.id())
            print(' pdg_code: \t \t \t ',            p.pdg_code())
            #print(' creation_process : \t \t ',      p.creation_process())
            #print(' nu_interaction_type: \t \t ',    p.nu_interaction_type())
            #print(' interaction_id: \t \t ',         p.interaction_id())
            #print(' nu_current_type: \t \t ',        p.nu_current_type())
            
            
            print(' group_id: \t \t \t ',            p.group_id())
            print(' children_id: \t \t \t ',         p.children_id())
            #print(' shape: \t \t \t ',               p.shape())
            print(' energy_init: \t \t \t ',         p.energy_init())
            print(' energy_deposit: \t \t ',         p.energy_deposit())
            print(' num_voxels: \t \t \t ',          p.num_voxels())

            print(' first_step: \t \t \t  (',        p.first_step().x(), ' , ', p.first_step().y(), ' , ', p.first_step().z(), ')')
            print(' last_step: \t \t \t  (',         p.last_step().x(), ' , ', p.last_step().y(), ' , ', p.last_step().z(), ')')
            #print(' end_position: \t \t \t  (',      p.end_position().x(), ' , ', p.end_position().y(), ' , ', p.end_position().z(), ')')
            #print(' distance_travel: \t \t ',        p.distance_travel())
            '''
            '''
            print(' momentum: ', p.momentum().x())
            print(' p: \t \t \t \t ',                p.p())
            print(' px: \t \t \t \t ',               p.px())
            print(' py: \t \t \t \t ',               p.py())
            print(' pz: \t \t \t \t ',               p.pz())
            print(' ------------------------------------------- ')

            print(' vertex position: \t \t  (',      p.position().x(), ' , ', p.position().y(), ' , ', p.position().z(), ')')
            print(' vertex t: \t \t \t ',            p.t())
            print(' vertex x: \t \t \t ',            p.x())  # corresponds to the pi0 -> gamma+gamma vertex
            print(' vertex y: \t \t \t ',            p.y())  # corresponds to the pi0 -> gamma+gamma vertex
            print(' vertex z: \t \t \t ',            p.z())  # corresponds to the pi0 -> gamma+gamma vertex
            #print('--------------------------------------------------------------------------------------------------------')
            '''

            if p.parent_track_id() not in pi0_track_ids:
                n_pi0_in_event += 1
                pi0_track_ids.append(p.parent_track_id())
                pi0_pdg_code.append(p.parent_pdg_code())
                gamma_1_track_id.append(p.track_id())
                gamma_1_pdg_code.append(p.pdg_code())
                gamma_1_mom.append([p.px(),p.py(),p.pz()])
                gamma_1_first_step.append([p.first_step().x(),p.first_step().y(),p.first_step().z()])
                gamma_1_pos.append([p.x(),p.y(),p.z()])  # corresponds to the pi0 -> gamma+gamma vertex
                gamma_1_ekin.append(p.energy_init())     # initial energy of photon, = np.sqrt(p.px()**2+p.py()**2+p.pz()**2)
                gamma_1_energy_deposit.append(p.energy_deposit())
                gamma_1_num_voxels.append(p.num_voxels())
            else:
                # check whether ther pi0 track id corresponds to the latest one (in order to not assign the photon to a wrong parent)
                if p.parent_track_id() == pi0_track_ids[len(pi0_track_ids)-1]:
                    #pi0_track_ids.append(p.parent_track_id())
                    pi0_pdg_code.append(p.parent_pdg_code())
                    gamma_2_track_id.append(p.track_id())
                    gamma_2_pdg_code.append(p.pdg_code())
                    gamma_2_mom.append([p.px(),p.py(),p.pz()])
                    gamma_2_first_step.append([p.first_step().x(),p.first_step().y(),p.first_step().z()])
                    gamma_2_pos.append([p.x(),p.y(),p.z()])  # corresponds to the pi0 -> gamma+gamma vertex
                    gamma_2_ekin.append(p.energy_init())     # initial energy of photon, = np.sqrt(p.px()**2+p.py()**2+p.pz()**2)
                    gamma_2_energy_deposit.append(p.energy_deposit())
                    gamma_2_num_voxels.append(p.num_voxels())
                else:
                    print('WARNING: Assigning a gamma to the wrong parent ...')
    
    #print(' true_gammas_group_ids: ', true_gammas_group_ids)


    # Loop over all particles and build lists with track IDs produced by photons originated by neutral pion decays
    # ------------------------------------------------------
    if n_true_gammas > 0:
        particle_ids_gamma_showers = [[] for _ in range(n_true_gammas)] # particle IDs of all particles corresponding to a true shower (produced by photon from pi0 decay)
        track_ids_showers          = [[] for _ in range(n_true_gammas)] #    track IDs of all particles corresponding to a true shower (produced by photon from pi0 decay)
        true_pixel_sum             = [0. for _ in range(n_true_gammas)] # [MeV]
        true_showers_n_voxels      = [[] for _ in range(len(true_gammas_group_ids))] # list of arrays (number of arrays = number of true gammas from pi0 decays). Each array stores as entries: p.num_voxels() for every particle in the shower.
        used_particles             = [[] for _ in range(len(true_gammas_group_ids))] # to store the track_ids of all used particles to calc. the showers n_voxels
        counter = 0
        for particle in range(len(chain.event['particles'][0])):
            p = chain.event['particles'][0][particle]
            if p.parent_pdg_code() == 111 and p.pdg_code() == 22:
                particle_ids_gamma_showers[counter].append(p.id())
                track_ids_showers[counter].append(p.track_id())
                true_pixel_sum[counter] += p.energy_deposit() # Note: photons can have edep > 0: e+/e- pair production is recorded as a photon
                counter += 1
            if p.group_id() in true_gammas_group_ids:
                for index in range(len(true_gammas_group_ids)):
                    if true_gammas_group_ids[index] == p.group_id():
                        if index not in used_particles[index]:
                            true_showers_n_voxels[index].append(p.num_voxels())
                            #print(' voxels: ', p.num_voxels())
                            used_particles[index].append(p.track_id())
        '''
        print(' used_particles: ', used_particles)
        print(' true_showers_n_voxels: ', true_showers_n_voxels)
        '''
        

        '''
        print(' particle_ids_gamma_showers: ', particle_ids_gamma_showers)
        print(' track_ids_showers: ', track_ids_showers)
        print(' ---------------------------------------------------------------------- ')
        '''

        for particle in range(len(chain.event['particles'][0])):
            p = chain.event['particles'][0][particle]
            for gamma in range(n_true_gammas):
                if p.parent_id() in particle_ids_gamma_showers[gamma] and p.id() not in particle_ids_gamma_showers[gamma]:
                    particle_ids_gamma_showers[gamma].append(p.id())
                    true_pixel_sum[gamma] += p.energy_deposit()
                    track_ids_showers[gamma].append(p.track_id())
                    '''
                    print(' \n ---------------------- PHOTON Nr.', gamma, '---------------------- ')
                    print(' parent id: \t \t ',    p.parent_id())
                    print(' parent track id: \t ', p.parent_track_id())
                    print(' parent pdg: \t \t ',   p.parent_pdg_code())
                    print(' id: \t \t \t ',        p.id())
                    print(' track id: \t \t ',     p.track_id())
                    print(' pdg: \t \t \t ',       p.pdg_code())
                    print(' edep: \t \t \t ',      p.energy_deposit())
                    print(' \n particle_ids_gamma_showers[', gamma, ']: \n', particle_ids_gamma_showers[gamma])
                    print(' \n track_ids_showers[', gamma, ']: \n', track_ids_showers[gamma])
                    print(' \n pixel sums: \n', true_pixel_sum)
                    '''
        '''
        print(' particle_ids_gamma_showers: ', particle_ids_gamma_showers)
        print(' track_ids_showers: ', track_ids_showers)
        print(' pixel sum: ', true_pixel_sum)
        '''



    #print(' ====================== ============== ======================')
    #print(' ====================== RECONSTRUCTION ======================')
    #print(' ====================== ============== ======================')
    # For all reconstructed showers (can much more than true showers)
    reco_showers_start_pos = []
    reco_showers_dir       = []
    reco_showers_voxels    = []
    reco_showers_energy    = []
    reco_showers_pid       = []

    # For each true shower only, assign the reconstructed shower which has reco start point closest to the true shower
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # POSSIBLE POINT OF FAILURE: IF n_reco_showers < n_true_showers
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    closest_reco_showers_first_step = np.zeros((n_true_gammas, 3))
    closest_reco_showers_dir        = np.zeros((n_true_gammas, 3))
    closest_reco_showers_n_voxels   = np.zeros((n_true_gammas, 1))
    closest_reco_showers_energy     = np.zeros((n_true_gammas, 1))

    
    # Loop over all (reconstructed) showers:
    # ------------------------------------------------------
    showers = chain.output['showers']
    n_reco_gammas = len(showers)
    for sh in range(len(showers)):
        reco_showers_start_pos.append(np.array(showers[sh].start))
        reco_showers_dir.      append(np.array(showers[sh].direction))
        reco_showers_voxels.   append(np.array(showers[sh].voxels))
        reco_showers_energy.   append(showers[sh].energy)
        reco_showers_pid.      append(showers[sh].pid)
        '''
        #print('start: \t ',    np.array(showers[sh].start))
        #print('dir: \t ',      np.array(showers[sh].direction))
        #print('voxels: ',      showers[sh].voxels)
        #print('voxels: ',      showers[sh].voxels[0:10])
        #print('n voxels: ',    len(showers[sh].voxels))
        #print('energy: \t ',   showers[sh].energy)
        #print('pid: \t ',      showers[sh].pid)
        #print(' ------------------------------------------- ')
        '''


    # Get true gamma's directions in order to associate the reconstructed showers to the true
    # ------------------------------------------------------
    true_photons_first_steps = []
    true_photons_energies = []
    for particle in range(len(chain.event['particles'][0])):
        p = chain.event['particles'][0][particle]
        if p.parent_pdg_code() == 111 and p.pdg_code() == 22:
            true_photons_first_steps.append(np.array([p.first_step().x(),p.first_step().y(),p.first_step().z()]))
            true_photons_energies.append(np.sqrt(p.px()**2+p.py()**2+p.pz()**2))
    '''
    #print(' true_photons_first_steps: \n ', true_photons_first_steps)
    #print(' true_photons_energies [MeV]: ', true_photons_energies)
    '''


    # Get the distances between the reconstructed shower's starting points
    # (note: each group is considered as a single shower)
    # and the true gamma's start points (first_step).
    # Calculate the pairwise distances between reco start and true start positions -> yields matrix.
    # Assign true shower to reco shower by taking the first minimum from the matrix,
    # then the second minimum, and so on.
    # First, create the matrix with the pairwise distances, which
    # is a matrix of dimensions N_true_showers x N_reco_showers)
    # Then, sum the energy deposits in each group and compare it with the true photon's energy.
    # Note: could do it also with the true group's start point
    # -> look for the gamma which produced this group (closest true gamma end point).
    # ------------------------------------------------------
    distances = np.zeros((len(true_photons_first_steps), len(reco_showers_start_pos)))
    for true_sh in range(len(true_photons_first_steps)):
        minimum_temp = 999.9
        for reco_sh in range(len(reco_showers_start_pos)):
            distances[true_sh][reco_sh] = linalg.norm(true_photons_first_steps[true_sh]-reco_showers_start_pos[reco_sh])
            '''
            #print(' distances: ', distances[true][reco])
            #print(' minimum_temp: ', minimum_temp)
            '''

    # Get the minima of the distances matrix
    # ------------------------------------------------------
    for true_gamma in range(n_true_gammas):
        if n_reco_gammas > 0:
            a, b = np.where(distances == np.min(distances)) # from the whole 2D array, pick the smallest value
            #print(' a: ', a[0])
            #print(' b: ', b[0])
            distances[a[0],:] = 9999.9 # set the row of the picked value to a large number in order not to be selected a second time
            distances[:,b[0]] = 9999.9 # also for the columns

            # assign the closest shower the correct reconstructed shower's variables
            try:
                closest_reco_showers_first_step[a[0],:] = reco_showers_start_pos[b[0]]
                closest_reco_showers_dir[a[0],:]        = reco_showers_dir[b[0]]
                closest_reco_showers_n_voxels[a[0],:]   = len(reco_showers_voxels[b[0]])
                closest_reco_showers_energy[a[0],:]     = reco_showers_energy[b[0]]
            except:
                print(' WARNING: Cannot assign reco shower to a true shower. Pass ... ')
                print(' Fix this problem some point!! ')
                pass
    '''
    #print(' closest_reco_showers_fist_step: ',   closest_reco_showers_fist_step)
    #print(' closest_reco_showers_dir: ',         closest_reco_showers_dir)
    #print(' closest_reco_showers_n_voxels[0]: ', closest_reco_showers_n_voxels[0])
    #print(' closest_reco_showers_n_voxels[1]: ', closest_reco_showers_n_voxels[1])
    '''

    # Calculate overlap between true and reco voxels for each shower
    # ------------------------------------------------------
    if n_true_gammas > 0:
        for index in range(len(true_showers_n_voxels)):
            '''
            #print(' true_showers_n_voxels[', index, '] ', true_showers_n_voxels[index]) # array with n_voxels of each particle in the true shower
            print(' true showers voxel count[', index, ']: ', np.sum(true_showers_n_voxels[index]))
            print(' reco showers voxel count[', index, ']: ', np.sum(closest_reco_showers_n_voxels[index]))
            '''
    
    

    #print(' ====================== ============== ======================')
    #print(' ======================  WRITE 2 FILE  ======================')
    #print(' ====================== ============== ======================')
    # Write data to CSV file
    for pi0 in range(len(pi0_track_ids)):
        #print(' len(pi0_track_ids): ', len(pi0_track_ids))
        #print(' pi0: ', pi0)
        try:
            log_true_pi0.record(['event','n_pi0_in_event','n_true_gammas_in_event','n_reco_gammas_in_event',
                                 'true_pi0_track_id','true_pi0_pdg_code',
                                 'true_gamma_1_track_id','true_gamma_1_pdg_code',
                                 'true_gamma_1_pos_x','true_gamma_1_pos_y','true_gamma_1_pos_z',  # vertex pi0 -> gamma + gamma
                                 'true_gamma_1_mom_x','true_gamma_1_mom_y','true_gamma_1_mom_z',
                                 'true_gamma_1_ekin','true_gamma_1_energy_deposit',
                                 'true_gamma_1_pixel_sum','true_gamma_1_num_voxels',
                                 'true_gamma_1_first_step_x','true_gamma_1_first_step_y','true_gamma_1_first_step_z',
                                 'true_gamma_2_track_id','true_gamma_2_pdg_code',
                                 'true_gamma_2_pos_x','true_gamma_2_pos_y','true_gamma_2_pos_z',
                                 'true_gamma_2_mom_x','true_gamma_2_mom_y','true_gamma_2_mom_z',
                                 'true_gamma_2_ekin','true_gamma_2_energy_deposit',
                                 'true_gamma_2_pixel_sum','true_gamma_2_num_voxels',
                                 'true_gamma_2_first_step_x','true_gamma_2_first_step_y','true_gamma_2_first_step_z',

                                 'reco_gamma_1_first_step_x','reco_gamma_1_first_step_y','reco_gamma_1_first_step_z',
                                 'reco_gamma_1_dir_x','reco_gamma_1_dir_y','reco_gamma_1_dir_z',
                                 'reco_gamma_1_energy','reco_gamma_1_num_voxels',
                                 'reco_gamma_2_first_step_x','reco_gamma_2_first_step_y','reco_gamma_2_first_step_z',
                                 'reco_gamma_2_dir_x','reco_gamma_2_dir_y','reco_gamma_2_dir_z',
                                 'reco_gamma_2_energy','reco_gamma_2_num_voxels',
                                 'reco_pi0_mass'
                                ],
                                [ev,n_pi0_in_event,n_true_gammas,n_reco_gammas,
                                 pi0_track_ids[pi0],pi0_pdg_code[pi0],
                                 gamma_1_track_id[pi0],gamma_1_pdg_code[pi0],
                                 gamma_1_pos[pi0][0],gamma_1_pos[pi0][1],gamma_1_pos[pi0][2],  # vertex pi0 -> gamma + gamma
                                 gamma_1_mom[pi0][0],gamma_1_mom[pi0][1],gamma_1_mom[pi0][2],
                                 gamma_1_ekin[pi0],gamma_1_energy_deposit[pi0],
                                 true_pixel_sum[pi0],gamma_1_num_voxels[pi0],
                                 gamma_1_first_step[pi0][0],gamma_1_first_step[pi0][1],gamma_1_first_step[pi0][2],
                                 gamma_2_track_id[pi0],gamma_2_pdg_code[pi0],
                                 gamma_2_pos[pi0][0],gamma_2_pos[pi0][1],gamma_2_pos[pi0][2],
                                 gamma_2_mom[pi0][0],gamma_2_mom[pi0][1],gamma_2_mom[pi0][2],
                                 gamma_2_ekin[pi0],gamma_2_energy_deposit[pi0],
                                 true_pixel_sum[pi0+1],gamma_1_num_voxels[pi0],
                                 gamma_2_first_step[pi0][0],gamma_2_first_step[pi0][1],gamma_2_first_step[pi0][2],

                                 closest_reco_showers_first_step[pi0][0],closest_reco_showers_first_step[pi0][1],closest_reco_showers_first_step[pi0][2],
                                 closest_reco_showers_dir[pi0][0],closest_reco_showers_dir[pi0][1],closest_reco_showers_dir[pi0][2],
                                 closest_reco_showers_energy[pi0][0],closest_reco_showers_n_voxels[pi0][0],
                                 closest_reco_showers_first_step[pi0+1][0],closest_reco_showers_first_step[pi0+1][1],closest_reco_showers_first_step[pi0+1][2],
                                 closest_reco_showers_dir[pi0+1][0],closest_reco_showers_dir[pi0+1][1],closest_reco_showers_dir[pi0+1][2],
                                 closest_reco_showers_energy[pi0+1][0],closest_reco_showers_n_voxels[pi0+1][0],
                                 np.sqrt(2.*closest_reco_showers_energy[pi0][0]*closest_reco_showers_energy[pi0+1][0]*(1.-np.dot(closest_reco_showers_dir[pi0],closest_reco_showers_dir[pi0+1])))
                                ])
        except IndexError:
            print(' IndexError: Fix this problem at some point!! ')
        except ValueError:
            print(' ValueError: Fix this problem at some point!! ')
            pass
            # Note: Two gammas, originated from the same pi0 decay, have parent_ids which differ by 1 (e.g. 22 and 23).
            # In order to check whether the parent of the two gammas is the same, check for parent_track_id,
            # which should be the same for two gammas produced in the same pi0 decay.
        log_true_pi0.write()
        log_true_pi0.flush()

Welcome to JupyROOT 6.16/00
Initialized Pi0 mass chain, log path: pi0_chain_test_log.csv

Config processed at: Linux rberner-X1 5.3.0-28-generic #30~18.04.1-Ubuntu SMP Fri Jan 17 06:14:09 UTC 2020 x86_64 x86_64 x86_64 GNU/Linux

$CUDA_VISIBLE_DEVICES="None"

{   'iotool': {   'batch_size': 1,
                  'collate_fn': 'CollateSparse',
                  'dataset': {   'data_keys': [   '/home/rberner/cernbox/PhD/pi0_reconstruction/reco_software/data/train.root'],
                                 'limit_num_files': 25,
                                 'name': 'LArCVDataset',
                                 'schema': {   'cluster_label': [   'parse_cluster3d_full',
                                                                    'cluster3d_pcluster',
                                                                    'particle_corrected'],
                                               'input_data': [   'parse_sparse3d_scn',
                                                       

 event:  0
 event:  1
 event:  2
 event:  3
 event:  4


In [5]:
    # Draw the last event if requested
    # Note: matches: if there are two reco showers pointing to the same point AND
    # this point is close to a track

    #if 'matches' in chain.output and len(chain.output['matches']) and i_draw < n_draw:
    #    chain.draw()
    #    i_draw += 1

    out.update(progress(ev,data_size,'images'))

out.update(progress(data_size,data_size,'images'))

# Simplest way to run the full chain
#chain = Pi0Chain(io_cfg, chain_cfg)
#chain.run()


In [6]:
'''
import numpy as np
import pandas as pd
import seaborn
from matplotlib import pyplot as plt

seaborn.set(rc={'figure.figsize':(15, 10),})
seaborn.set_context('talk') # or paper

def gaus(x, a, mu, sigma):
    return  a*np.exp(-(x-mu)**2/2./sigma**2)/np.sqrt(2./np.pi)/sigma

def fit_func(bins, n, func):
    from scipy.optimize import curve_fit
    center = (bins[:-1] + bins[1:]) / 2
    popt, pcov = curve_fit(func, center, n, p0=(100, 100, 10))
    print(" Fitted parameters: \n a [-]: \t ", popt[0],
          " \n \u03BC [MeV/c2]: \t ", popt[1],
          " \n \u03C3 [MeV/c2]: \t ", popt[2])

    x = np.arange(0, 300, 1)
    y = func(x, popt[0], popt[1], popt[2])
    plt.plot(x, y, label='Fit: mass=%5.3f, width=%5.3f' % (popt[1], popt[2]))
    plt.legend()

# Load the output file, draw mass peak
#df = pd.read_csv(chain_cfg['name']+'_log.csv')
df = pd.read_csv('true_pi0s.csv')
plt.figure()
fig, ax = plt.subplots()
n, bins, patches = ax.hist(df.E_kin, bins=60, range=[0,300], alpha=0.75)
#np.sqrt(df.px_MeV**2+df.py_MeV**2+df.pz_MeV**2)
ax.set_xlabel('$\pi^0$ $E_{kin}$ [MeV]')
#ax.set_xlabel('Invariant $\pi^0$ mass [MeV/c$^2$]')

# Fit the peak with a Gaussian
fit_func(bins, n, gaus)
plt.show()

# Print pion mass dataframe
df[35:45]
#print(df.to_string())
'''

'\nimport numpy as np\nimport pandas as pd\nimport seaborn\nfrom matplotlib import pyplot as plt\n\nseaborn.set(rc={\'figure.figsize\':(15, 10),})\nseaborn.set_context(\'talk\') # or paper\n\ndef gaus(x, a, mu, sigma):\n    return  a*np.exp(-(x-mu)**2/2./sigma**2)/np.sqrt(2./np.pi)/sigma\n\ndef fit_func(bins, n, func):\n    from scipy.optimize import curve_fit\n    center = (bins[:-1] + bins[1:]) / 2\n    popt, pcov = curve_fit(func, center, n, p0=(100, 100, 10))\n    print(" Fitted parameters: \n a [-]: \t ", popt[0],\n          " \n μ [MeV/c2]: \t ", popt[1],\n          " \n σ [MeV/c2]: \t ", popt[2])\n\n    x = np.arange(0, 300, 1)\n    y = func(x, popt[0], popt[1], popt[2])\n    plt.plot(x, y, label=\'Fit: mass=%5.3f, width=%5.3f\' % (popt[1], popt[2]))\n    plt.legend()\n\n# Load the output file, draw mass peak\n#df = pd.read_csv(chain_cfg[\'name\']+\'_log.csv\')\ndf = pd.read_csv(\'true_pi0s.csv\')\nplt.figure()\nfig, ax = plt.subplots()\nn, bins, patches = ax.hist(df.E_kin, bins

# Questions

In [7]:
chain.event.keys()

dict_keys(['input_data', 'segment_label', 'cluster_label', 'particles', 'ppn_label', 'index'])

In [8]:
chain.output.keys()
# Note: The chain produces an output object (chain.output),
# which can be thought of as the reconstructed quantities.

dict_keys(['cluster_label', 'charge', 'segment', 'shower_mask', 'energy', 'showers'])

In [9]:
input_data = chain.event['input_data']
print(input_data[0:5])  # QUESTION: TRUE voxel_index_x, voxel_index_y, voxel_index_z, batch, energy_depostion (or charge?)
print(input_data.shape) # QUESTION: 6065 = number of energy deposits in this event?

[[2.27000000e+02 4.70000000e+01 0.00000000e+00 0.00000000e+00
  5.27665079e-01]
 [3.33000000e+02 3.32000000e+02 0.00000000e+00 0.00000000e+00
  4.69809949e-01]
 [2.27000000e+02 4.70000000e+01 1.00000000e+00 0.00000000e+00
  1.91275984e-01]
 [2.27000000e+02 4.80000000e+01 1.00000000e+00 0.00000000e+00
  3.14670533e-01]
 [3.33000000e+02 3.31000000e+02 1.00000000e+00 0.00000000e+00
  4.27937686e-01]]
(3835, 5)


In [10]:
# QUESTION: How is the voxel mapping done?

In [11]:
segment_label = chain.event['segment_label']
print(segment_label[0:5]) # QUESTION: TRUE voxel_index_x, voxel_index_y, voxel_index_z, batch, true semantic class
print(segment_label.shape) # QUESTION: 6065 = number of energy deposits in this event?

[[227.  47.   0.   0.   1.]
 [333. 332.   0.   0.   1.]
 [227.  47.   1.   0.   1.]
 [227.  48.   1.   0.   1.]
 [333. 331.   1.   0.   1.]]
(3835, 5)


In [24]:
cluster_label = chain.event['cluster_label']
print(cluster_label[0:5]) # TRUE? x, y, z, batch_id, voxel_value, cluster_id, group_id, semantic type
print(cluster_label.shape)

[[5.67000000e+02 5.50000000e+02 4.86000000e+02 0.00000000e+00
  2.20694482e-01 0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [5.67000000e+02 5.50000000e+02 4.87000000e+02 0.00000000e+00
  1.87287438e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [5.67000000e+02 5.50000000e+02 4.88000000e+02 0.00000000e+00
  9.27632153e-01 0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [5.67000000e+02 5.51000000e+02 4.88000000e+02 0.00000000e+00
  9.78750348e-01 0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [5.67000000e+02 5.51000000e+02 4.89000000e+02 0.00000000e+00
  1.63090372e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]
(3893, 8)


In [13]:
cluster_label = chain.output['cluster_label']
print(cluster_label[0:5])
print(cluster_label.shape)
# QUESTION: The difference between chain.event and chain.output is TRUE vs. RECO, right?

[[5.67000000e+02 5.50000000e+02 4.86000000e+02 0.00000000e+00
  2.20694482e-01 0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [5.67000000e+02 5.50000000e+02 4.87000000e+02 0.00000000e+00
  1.87287438e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [5.67000000e+02 5.50000000e+02 4.88000000e+02 0.00000000e+00
  9.27632153e-01 0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [5.67000000e+02 5.51000000e+02 4.88000000e+02 0.00000000e+00
  9.78750348e-01 0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [5.67000000e+02 5.51000000e+02 4.89000000e+02 0.00000000e+00
  1.63090372e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]
(3611, 8)


In [14]:
charge = chain.output['charge'] # QUESTION: voxel_index_x, voxel_index_y, voxel_index_z, batch?, charge deposition?
print(charge[0:5])
print(charge.shape)

[[2.27000000e+02 4.70000000e+01 0.00000000e+00 0.00000000e+00
  5.27665079e-01]
 [3.33000000e+02 3.32000000e+02 0.00000000e+00 0.00000000e+00
  4.69809949e-01]
 [2.27000000e+02 4.70000000e+01 1.00000000e+00 0.00000000e+00
  1.91275984e-01]
 [2.27000000e+02 4.80000000e+01 1.00000000e+00 0.00000000e+00
  3.14670533e-01]
 [3.33000000e+02 3.31000000e+02 1.00000000e+00 0.00000000e+00
  4.27937686e-01]]
(3835, 5)


In [15]:
segment = chain.output['segment'] # QUESTION: Same as 'charge', but with semantic class?
print(segment[0:5])
print(segment.shape)

[[227.  47.   0.   0.   1.]
 [333. 332.   0.   0.   1.]
 [227.  47.   1.   0.   1.]
 [227.  48.   1.   0.   1.]
 [333. 331.   1.   0.   1.]]
(3835, 5)


In [16]:
shower_mask = chain.output['shower_mask'] # QUESTION: Contains voxel_indices with non-zero RECO charge?
print(shower_mask)
print(len(shower_mask[0]))

(array([ 180,  181,  187,  188,  189,  190,  191,  192,  200,  201,  202,
        203,  204,  205,  211,  219,  225,  226,  231,  232,  239,  240,
        511,  512,  513,  514,  522,  523,  524,  525,  534,  535,  536,
        537,  538,  539,  540,  541,  542,  550,  551,  552,  553,  554,
        564,  565,  566,  567,  576,  577,  578,  579,  580,  581,  588,
        589,  590,  591,  592,  593,  602,  603,  604,  605,  606,  617,
        618,  619,  620,  621,  630,  631,  632,  633,  634,  635,  644,
        645,  646,  647,  648,  656,  657,  658,  659,  660,  661,  662,
        670,  671,  672,  673,  674,  675,  676,  677,  678,  679,  680,
        690,  691,  692,  693,  694,  703,  704,  705,  706,  707,  718,
        719,  720,  721,  729,  730,  731,  732,  733,  741,  742,  743,
        744,  754,  755,  756,  757,  766,  767,  768,  769,  770,  779,
        780,  781,  782,  792,  793,  794,  795,  796,  797,  806,  807,
        808,  811,  812,  813,  814,  815,  816,  

In [17]:
energy = chain.output['energy'] # QUESTION: Same as 'charge', but with semantic class?
print(energy[0:5])
print(energy.shape)

[[2.27000000e+02 4.70000000e+01 0.00000000e+00 0.00000000e+00
  5.27665079e-01]
 [3.33000000e+02 3.32000000e+02 0.00000000e+00 0.00000000e+00
  4.69809949e-01]
 [2.27000000e+02 4.70000000e+01 1.00000000e+00 0.00000000e+00
  1.91275984e-01]
 [2.27000000e+02 4.80000000e+01 1.00000000e+00 0.00000000e+00
  3.14670533e-01]
 [3.33000000e+02 3.31000000e+02 1.00000000e+00 0.00000000e+00
  4.27937686e-01]]
(3835, 5)


# Tests

In [18]:
showers = chain.output['showers']
#help(showers[0])
print(showers[0].start)
#print(showers[0].direction)
#print(showers[0].voxels)
#print(showers[0].energy)
#print(showers[0].pid)

[642.28557326 659.98881519  36.98107797]


In [19]:
index = chain.event['index']
print(index) # index of the event within the chain

[4]


In [20]:
particles = chain.event['particles'][0]
print(particles[0]) # QUESTION: entries?
#print(particles.shape) # QUESTION: entries?

<ROOT.larcv::Particle object at 0x949cbb0>


In [21]:
charge = chain.output['charge'][0:10]
#help(charge)
#print(charge)

In [22]:
energy = chain.output['energy'][0:10]
#help(energy)
#print(energy)

In [28]:
vertices = chain.output['vertices']
vertex = chain.output['vertices'][0]
#print(vertex)

KeyError: 'vertices'

In [ ]:
masses = chain.output['masses']
#print(masses)

In [ ]:
matches = chain.output['matches']
print(matches)

In [ ]:
segments = chain.event['segment_label']
print(segments[0:5]) # shows (true?): x, y, z, batch, semantic
print(segments.shape)

In [ ]:
cluster_label = chain.output['cluster_label']
#print(cluster_label[0])

In [ ]:
charge = chain.output['charge']
#print(charge)

In [ ]:
shower = chain.output['showers'][0]
#help(shower)
#print(shower)

In [ ]:
#help(chain.event['particles'][0][particle].first_step())

In [ ]:
# The io-config files contain information like 'particles', 'cluster_label', etc.

# Get particle:
particle = chain.event['particles'][0][0]
#help(particle)
#help(particle.boundingbox_2d())

#cluster = chain.event['cluster_label'][0]
#print("cluster: ", cluster)
# to see what the entries of this array are, look up the information from the .cfg:
      #cluster_label:
      #  - parse_cluster3d_full
      #  - cluster3d_pcluster
      #  - particle_corrected
# -> Need to have a look to 'parse_cluster3d_full, which lives in mlreco/iotools/parse_cluster3d_full'

In [ ]:
# Initialize the chain
from pi0.chain import Pi0Chain
chain = Pi0Chain(io_cfg, chain_cfg)
cluster = chain.event['cluster_label']
help(cluster)

In [ ]:
# Note: lartpc_mlreco3d/mlreco/iotools/parser.py -> parse_cluster3d)
# The batch_id is not listed in the parser, but it is always added after x,y,z.
# Several clusters can form a single group (the group corresponds to the shower)
cluster_label = chain.event['cluster_label']
print(cluster_label[0:5]) # TRUE? x, y, z, batch_id, voxel_value, cluster_id, group_id, semantic type
print(cluster_label.shape)

# To loop over all clusters in the event:
#for cluster in range(len(chain.event['cluster_label'])):
    #c = chain.event['cluster_label'][cluster]
    #print(c)
    #print(' x: \t ', c[0], ' y: \t ', c[1], ' z: \t ', c[2], ' c_id: \t ', c[5], ' \t g_id: \t ', c[6])

# To obtain the group energies of a single event:
#c = chain.event['cluster_label']
#group_energies = [np.sum(c[c[:,6] == g][:,4]) for g in np.unique(c[:,6])]
#print(' group energies [MeV]: \t ', group_energies)